In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime

pełną dokumentację funkcji zrobi się później

In [2]:
t = 10000 ##żeby wykluczyć mało znane imiona i nazwiska

##imiona męskie i kobiece
im = pd.read_csv("IM.csv")
im = im[im["LICZBA WYSTĄPIEŃ"] > t] 
ik = pd.read_csv("IK.csv")
ik = ik[ik["LICZBA_WYSTĄPIEŃ"] > t]
ik = ik["IMIĘ_PIERWSZE"]
im = im["IMIĘ PIERWSZE"]

##nazwiska męskie i kobiece
nm = pd.read_csv("NM.csv")
nm = nm[nm["Liczba"] > t]
nk = pd.read_csv("NK.csv")
nk = nk[nk["Liczba"] > t]
nk = nk["Nazwisko aktualne"]
nm = nm["Nazwisko aktualne"]

ulice = pd.read_csv("ULIC.csv", sep=";")
ulice_jaworze = ulice.query('WOJ == 24 & POW == 2 & GMI == 6 & RODZ_GMI == 2') ##bo kod terytorialny jaworza to 24.2.6.2
ulice_jaworze = np.array(ulice_jaworze["NAZWA_1"]) ##i bierzemy ulice

##gry
gry = pd.read_csv("GRY.csv")
gry

,ID,NAZWA,RODZAJ,CENA,CZAS_GRY,MIN_GRACZY,MAX_GRACZY,MIN_WIEK,TURNIEJOWE
0,1,Brass: Birmingham,"ekonomiczna, strategiczna",250,150,2,4,14,0
1,2,Star Wars: Rebelia,"strategiczna, science-fiction",390,210,2,4,14,0
2,3,Wojna o Pierścień: Druga edycja,"przygodowa, fantasy",330,150,2,4,13,0
3,4,Zaginiona Wyspa Arnak,"przygodowa, eksploracyjna",150,90,1,4,12,0
4,5,Zamki Burgundii: BIG BOX,strategiczna,140,60,2,4,12,0
...,...,...,...,...,...,...,...,...,...
95,96,Eurobusiness,"ekonomiczna, rywalizacyjna",30,90,2,5,14,0
96,97,Boop,"logiczna, rywalizacyjna",160,20,2,2,10,0
97,98,Mordercze krewetki,"karciane, towarzyskie",80,10,2,6,7,0
98,99,The binding of Isaac: Four souls,"przygodowa, karciana",190,45,2,4,13,0


In [3]:
def ludzie(mi, ki, mn, kn, ul, pk= 0.516, N=7349 ):
    """
    Funkcja do generowania ludności z Jaworza
    pk = 0.516 - procent kobiet w społeczeństwie
    N = 7349 - (w miarę) aktualny stan ludności w tej urokliwej wsi
    
    """
    
    imie = []
    nazwisko = []
    wiek = []
    adres = []
    tel = []
    
    for n in range(N):
        
        p = random.random()
        
        if p < pk: ##losujemy kobietę
            imie.append(random.choice(ki).title()) ##title z AAAA robi Aaaa (a taką formę imion mamy)
            nazwisko.append(random.choice(kn).title())
        
        else: #losujemy mężczyznę
            imie.append(random.choice(mi).title())
            nazwisko.append(random.choice(mn).title())
        
        p = random.random()
        
        if p < 0.575: ##dane statystyczne - 57.5% osób jest w wieku produkcyjnym
            wiek.append(random.randint(18,60))
        elif p < 0.77:
            wiek.append(random.randint(8, 18)) ##19.4% w przedprodukcyjnym
        else:
            wiek.append(random.randint(60,99)) ##reszta starsza
            
            
        p = random.random()   
        if p< 0.90:
            adres.append(random.choice(ul) + ' ' + str(random.randint(1,100)))
            
            p = random.random()
            if p< 0.90:
                tel.append(random.randint(500000000, 999999999))
            else:
                tel.append("NULL")
                
        else:
            adres.append("NULL")
            tel.append(random.randint(500000000, 999999999))
        
        
            
    
    return pd.DataFrame({"id_mieszkańca": range(1,N+1), "imię": imie, "nazwisko": nazwisko, 
                         "wiek": wiek, "adres": adres, "telefon": tel})

In [4]:
def transform_wiek(wiek): 
    """
    Funkcja zwracająca częstość z jaką dana osoba na podstawie wieku mogłaby nas odwiedzać. 
    Im większa wartość tym większe prawdopodobieństwo wizyty.
    
    """
    
    if wiek < 10:
        return 1
    elif wiek < 12:
        return 2
    elif wiek < 16:
        return 3
    elif wiek < 20:
        return 5
    elif wiek < 30:
        return 6
    elif wiek < 38:
        return 5
    elif wiek < 45:
        return 4
    elif wiek < 55:
        return 3
    elif wiek < 70:
        return 2
    else:
        return 1


In [5]:
def inven(n = 100):
    """
    Funkcja tworząca prosty magazyn (wynajem/sklep). Zwraca ile gier o danym tytule (id_tytułu) jest u nas dostępna na półce.
    n = 100 - bo tyle różnych gier (i takich id) mamy w asortymencie
    """
    
    ilosc = np.random.randint(20, 30, n) ## od 20 do 30 każdego z tytułów
    ids = range(1, (n+1))
    ilosci = [ids[x] for x in range(len(ids)) for i in range(ilosc[x])]
    
    return pd.DataFrame({"id_inv": range(1, (len(ilosci)+1)),"id_tytuł": ilosci })

In [6]:
ludzie(im, ik, nm, nk, ulice_jaworze)

,id_mieszkańca,imię,nazwisko,wiek,adres,telefon
0,1,Kinga,Kozak,57,Polna 100,571892681
1,2,Olivia,Wojciechowska,43,Na Stoku 32,952561018
2,3,Eryk,Leszczyński,34,Zielona 37,753160890
3,4,Yuliia,Kozioł,48,Łubinowa 28,673127478
4,5,Wioletta,Pawlak,67,Górska 94,531307816
...,...,...,...,...,...,...
7344,7345,Zdzisława,Górecka,39,Zawilcowa 14,582177487
7345,7346,Maciej,Wysocki,29,Górska 26,821571707
7346,7347,Halyna,Krawczyk,97,Bratków 35,901661060
7347,7348,Helena,Sokołowska,24,Zielna 10,532292730


In [58]:
def odwiedziny(start = datetime.datetime(2022, 6, 6, 9, 0, 0), lamb = 4):
    """
    Funkcja symulująca odwiedziny w naszym sklepie - sprzedaże, wynajmy, a także zakup w outlecie. 
    Zwraca godziny wizyt, mieszkańców, co kupili/ wynajęli.
    start - dzień startowy naszego przedsiębiorstwa, niech to będzie 6 czerwca 2022, 9:00
    lamb - lambda - parametr potrzebny do generowania procesu Poissona - zakładamy 4 klientów na godzinę w naszym sklepie
   
    """
    
    mieszkancy = ludzie(im, ik, nm, nk, ulice_jaworze) #baza mieszkańców, którzy mogą nas odwiedzić
    T = 239/30 #7h 58 min bo tyle nasz sklep jest otwarty (z małym buforem, żeby nikt nie kupił czegoś po 1 s od otwarcia)
    
    now = datetime.datetime.now()
    days = (now - start).days ##żeby te daty były do dzisiaj
    
    
    inv_ren = inven() ##startowy magazyn wynajmu
    inv_shop = inven() ##startowy magazyn sklepu
    pp = [transform_wiek(mieszkancy["wiek"][x]) for x in range(len(mieszkancy))] ##wagi dla mieszkancow do losowania
    
    df = pd.DataFrame({}) ##podstawowa tabela, do której dodawane będą wizyty
    rental =  pd.DataFrame({"id_inv": [], "return_date": [], "zniszczona": []}) ##tabela pomocnicza dla inv rentalowego
    kupno =  pd.DataFrame({"id_inv": [], "kupno": [], "wizyta":[]}) ##dla kupna
    outlet = pd.DataFrame({"id_inv": [], "id_gry": [], "return_date": []}) ##dla outletu 
    arch_outlet =  pd.DataFrame({"id_inv": [], "date": []})
    
    swieta = [datetime.datetime(2023, 1, 6, 9, 0, 0), datetime.datetime(2023, 4, 9, 9, 0, 0), 
              datetime.datetime(2023, 4, 10, 9, 0, 0), datetime.datetime(2023, 3, 1, 9, 0, 0), 
              datetime.datetime(2023, 3, 3, 9, 0, 0), datetime.datetime(2023, 6, 8, 9, 0, 0), 
              datetime.datetime(2022, 6, 16, 9, 0, 0), datetime.datetime(2022, 8, 15, 9, 0, 0),
              datetime.datetime(2022, 11, 1, 9, 0, 0), datetime.datetime(2022, 11, 11, 9, 0, 0),
              datetime.datetime(2022, 12, 25, 9, 0, 0), datetime.datetime(2022, 12, 26, 9, 0, 0),]
    dd = [(s - start).days + 1 for s in swieta] ##dni jakie minęły od startu do święta (żeby móc je ominąć łatwo)
    
    exp = np.ceil(np.random.exponential(1, 150)) ##z tego będą losowane ilości gier kupionych/wynajętych
    
    for d in range(1, days + 1):
        
        if d%6 == 0 or d%7 == 0 or (d in dd): ## wyłączenie weekendów oraz świąt
            pass
        
        else:
            ##odwiedziny zgodne z procesem poissona
            N = np.random.poisson(T*lamb) ##ilość osób na cały dzień
            ts = sorted(T*np.random.uniform(0,1, N)) ##czasy wizyt (takie surowe)
            t_wizyty = [(start + datetime.timedelta(minutes=1) + datetime.timedelta(
                minutes=round(t*60,1))) for t in ts] #.strftime("%m/%d/%Y %H:%M:%S") for t in ts]
            ##formatka wizyt
            
            for t in t_wizyty:
                ##losowanie klientów z populacji mieszkańców
                klient = random.choices(mieszkancy["id_mieszkańca"], weights=pp , k=1)
            
                #losowanie sprzedawcy (zakładamy, że sprzedawca 1 i 2 obsługują kasy przez te 8 h)
                sprzedawca = random.choices([1,2], k=1)
            
                ##ktoś może chce kupić kilka gier zamiast jednej
                ilosc = random.choices(exp, k = 1)
            
                ##ktoś przychodzi z zamiarem kupna/wynajmu bądź skorzystania z outletu (jeśli może)
                ##ilość dostępnych gier w outlecie musi wynosić min tyle ile ktoś chce tych gier zakupić
                
                if len(outlet[outlet["return_date"] < start]) >= ilosc[0]: 
                    rodzaj = random.choices(["wynajem", "kupno", "outlet"], weights = [10, 10, 2], k = 1)
                    ##ustalamy prawdopodobieństwa skorzystania z outletu na 2:10:10 względem zwykłego kupna i wynajmu
                else:
                    rodzaj = random.choices(["wynajem", "kupno"], k = 1)
                    #jeśli outlet jest za słabo wyposażony zostaje tylko opcja wynajmu / kupna z wagami 1:1
                
                if rodzaj == ["wynajem"]:
                    
                    ##losuje zwrot za 1 lub 2 dni w godzinach otwarcia, bo zakładamy, że tyle mają czasu na zwrot
                    return_date = start + datetime.timedelta(days=random.randint(1,2)) + datetime.timedelta(
                        hours=round(random.uniform(1,T), 2))
                    
                    ##losuje id_gry, które ktoś wypożycza
                    ##szukamy tych id które są dostępne, czyli NIE ma ich w df rental z returndate > dziś
                    
                    not_in_use = np.array(rental[rental["return_date"] > start]["id_inv"]) ##niedostępne
                    ids = np.array(inv_ren["id_inv"]) ##wszystkie
                    in_use = np.setdiff1d(ids, not_in_use) ##różnica między wszystkimi a tymi niedostępnymi
                    gra = random.sample(sorted(in_use), k = int(ilosc[0])) ##z tych dostepnych losowanie gry
                    
                    ##ewentualne zniszczenia zwróconej gry - prawdopodobieństwo ustalamy na 1/100
                    fault = random.choices([0,1], weights = [99,1], k = int(ilosc[0])) 
                    
                    ##wrzucenie informacji do tymczasowej tabeli rental
                    rent_gry =  pd.DataFrame({"id_inv": gra, "return_date": np.repeat(return_date, int(ilosc[0])), 
                                              "zniszczona": fault})
                    rental = pd.concat([rental, rent_gry])
                    
                    ## gra może zostać zniszczona, a więc może trafić do outletu
                    if sum(fault) > 0: ##czy były jakieś zniszczenia - jeśli tak to update outletu
                        
                        idd = rent_gry[rent_gry["zniszczona"] == 1]["id_inv"] ##te zniszczone id_inv
                        
                        for i in idd: ##dla każdej zniszczonej gry
                            ##znajdujemy id_tytułu (bo działaliśmy na inv)
                            id_tyt = inv_ren.loc[inv_ren['id_inv'] == i]["id_tytuł"] 
                
                            f = pd.DataFrame({"id_inv": i, "id_gry": id_tyt, "return_date": return_date})
                            outlet = pd.concat([outlet, f]) ## i dodajemy gre do outletu
                    
                elif rodzaj == ["kupno"]: 
                    ##to samo co z rentalem, tylko patrzymy na id gry, które ktoś kupił DZISIAJ 
                    ##bo zakładamy, że kupiona dzisiaj gra, dostępna będzie ponownie jutro
                    
                    return_date = None
                    not_in_use2 = np.array(kupno[kupno["kupno"] == start]["id_inv"])
                    ids2 = np.array(inv_shop["id_inv"])
                    in_use2 = np.setdiff1d(ids2, not_in_use2)
                    gra = random.sample(sorted(in_use2), k = int(ilosc[0]))
                
                    kup_gry =  pd.DataFrame({"id_inv": gra, "kupno": np.repeat(start, int(ilosc[0])),
                                             "wizyta": np.repeat(t, int(ilosc[0]))})
                    kupno = pd.concat([kupno, kup_gry])
                    
                else: ##outlet
                    
                    return_date = None
                    in_use = outlet[outlet["return_date"] < start] ##patrzymy na te już zwrócone i dostepne gry
                    gra = random.sample(sorted(in_use["id_inv"]), k = int(ilosc[0])) ##losowanie z dostepnego outletu
                    ##id_inv odnoszą się na razie do magazynu wynajmu, ale później odnosić się będą do swojego id
                    ##to w formatce tabel
                    
                    for g in gra:
                        outlet = outlet.drop(outlet.index[outlet["id_inv"] == g])
                        ##usuwanie z outletu kupionych pozycji
                        arch = pd.DataFrame({"id_inv": g, "date": t}, index=[0])
                        arch_outlet = pd.concat([arch_outlet, arch])
                        
                ##dołączenie danego klienta do df z poprzednich dni
                day = pd.DataFrame({"id_mieszkańca": klient, "wizyta": t,
                                "sprzedawca": sprzedawca, "ilosc": int(ilosc[0]), "id_gry": str(gra)[1:-1], "rodzaj": rodzaj,
                                   "return_date": return_date})
                df = pd.concat([df, day])
            
        ##przejście z datą na kolejny dzień
        start = start + datetime.timedelta(days=1)
        
    return df, rental, kupno, outlet, inv_ren, inv_shop, mieszkancy, arch_outlet

In [59]:
def format_tabel(wizyty, gry = gry):
    """
    Funkcja służąca do formatowania otrzymanych wyników z funkcji odwiedziny() w taki sposób, żeby zwracała
    gotowe tabele, które trafią do bazy danych.
    wizyty - wynik z funkcji odwiedziny()
    gry - tabela z grami naszego wykonania
    
    """
    tabela_g = wizyty[0] ##tabela główna
    rental_arch = wizyty[1] ##archiwum wynajmu
    shop_arch = wizyty[2] ##archiwum sprzedaży
    outlet_akt = wizyty[3] ##aktualny outlet
    rent_inv = wizyty[4] ##inventory rental
    shop_inv = wizyty[5] ##inventory shop
    mieszkancy = wizyty[6] ##cała społeczność
    outlet_arch = wizyty[7] ## archiwum outletu
    
    tabela_g = tabela_g.rename(columns = {"sprzedawca":"id_pracownika"})
    ## ------------- TABELA KLIENCI -------------
    
    #szukamy unikatowych osób, które pojawiły się u nas w sklepie i ich ostatnią wizytę
    klient = tabela_g.drop_duplicates(subset=['id_mieszkańca'], keep="last")[["id_mieszkańca", "wizyta"]]
    klienci = pd.merge(klient, mieszkancy, how = "inner", on = ["id_mieszkańca"]) ##dopasowanie względem id_mieszkanca
    klienci["id_klienta"] = range(1, len(klienci) + 1) ##stworzenie id klienta
    
    klienci_mieszkancy = klienci[["id_klienta", "id_mieszkańca"]] ##tabela pomocnicza z id klienta dopasowanymi do mieszkańca
    
    klienci = klienci[["id_klienta", "wizyta", "imię", "nazwisko", "wiek", "adres", "telefon"]] #końcowa tabela klientów
    
    ## ---------------- TABELA SPICHLERZ_WYNAJEM ---------------
    ceny = gry[["ID","CENA"]]
    ceny = ceny.rename(columns = {"ID": "id_tytuł", "CENA": "cena"}) ##tymczasowa tabela z cenami
    ceny["cena_wynajem"] = np.round(0.15*ceny["cena"], 2 ) ###bo ustalamy, że cena wynajmu gry kosztuje 15% jej ceny rynkowej
    
    one = pd.merge(rent_inv, ceny, on="id_tytuł") ##krok pierwszy - połączenie wynajmu z cenami najmu
    ids = np.array(rental_arch["id_inv"])
    rental_arch["id_inv"] = [int(i) for i in ids]  ##po random sample zostały float jako typ id, więc wracam na int
    two = rental_arch.drop_duplicates(subset=['id_inv'], keep='last') ##biorę pod uwagę tylko ostatnie wypożyczenie danej gry
    
    spichlerz_wynajem = pd.merge(one,
                             two[["id_inv","return_date"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_wynajem", "id_tytuł": "id_gry", "return_date": "ostatni_update"})
    ##i ostateczna tabela:
    spichlerz_wynajem = spichlerz_wynajem[["id_spichlerz_wynajem", "id_gry", "cena_wynajem", "ostatni_update"]]
    
    ## ------------------- TABELA SPICHLERZ_SKLEP ---------------
    
    ceny["cena_kupno"] = np.round(1.1*ceny["cena"], 2 ) ###bo ustalamy marżę 10% na zakupie
    raz = pd.merge(shop_inv, ceny, on="id_tytuł")[["id_tytuł", "cena_kupno", "id_inv"]] ##znowu pierwsze merge z cenami
    
    ids2 = np.array(shop_arch["id_inv"])
    shop_arch["id_inv"] = [int(i) for i in ids2] ##znowu zmiana na int
    shop_arch["kupno"] = [d + datetime.timedelta(days=1) for d in shop_arch["kupno"]] ## bo gra jest dostepna 1 dzień po zakupie
    dwa = shop_arch.drop_duplicates(subset=['id_inv'], keep='last') ##znowu patrzymy na ostatnie zakupy
    spichlerz_sklep = pd.merge(raz,
                             dwa[["id_inv","kupno"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_sklep", "id_tytuł": "id_gry", "kupno": "ostatni_update"})
    spichlerz_sklep = spichlerz_sklep[["id_spichlerz_sklep", "id_gry", "cena_kupno", "ostatni_update"]] ##ostatnia formatka
    
    ## ---------------- TABELA WYNAJEM ---------
    sub = pd.merge(rental_arch, tabela_g, on = ["return_date"])[["id_inv", "wizyta", "return_date", "zniszczona", "id_mieszkańca", "id_pracownika"]]
    ##łączymy archiwum z główną tabelą - mamy teraz więcej informacji i dodajemy id klienta
    wynajemt = pd.merge(sub, klienci_mieszkancy, how = "left", on = ["id_mieszkańca"]).rename(columns={"id_inv":"id_spichlerz_wynajem"})
    ## i merge tabeli tymczasowej ze spichlerzem, żeby uzyskać cenę
    wynajem = pd.merge(wynajemt, spichlerz_wynajem[["id_spichlerz_wynajem","cena_wynajem"]], on=["id_spichlerz_wynajem"], how="left")
    ##i ostateczna forma tabeli:
    wynajem =  wynajem.sort_values(by = "wizyta", inplace=False, ascending=True) ##sort po datach
    wynajem = wynajem.rename(columns = {"wizyta": "data_wynajmu", "return_date": "data_zwrotu"}) ##szybki rename
    wynajem["id_transakcji_wynajem"] = range(1, len(wynajem) + 1) ##dodanie id_transakcji
    wynajem = wynajem[["id_transakcji_wynajem", "id_spichlerz_wynajem", "data_wynajmu", "data_zwrotu", "id_pracownika", "id_klienta", "zniszczona"]]
    
    ## ----------- TABELA SKLEP --------------------
    
    shop_arch['wizyta'] = pd.to_datetime(shop_arch['wizyta']) ##merge z tabelą główną
    sklept = pd.merge(shop_arch, tabela_g, on="wizyta", how="left")[["id_inv", "wizyta", "id_mieszkańca", "id_pracownika"]]
    sklep = pd.merge(sklept, klienci_mieszkancy, how="left", on = ["id_mieszkańca"]).rename(columns = {"id_inv": "id_spichlerz_sklep"})
    sklep["id_transakcji_sklep"] = range(1, len(sklep) + 1)
    sklep = sklep[["id_transakcji_sklep", "id_spichlerz_sklep", "wizyta", "id_klienta", "id_pracownika"]]
    
    
    ## -------------- TABELA OUTLET ----------------
    
    outlet_arch['date'] = pd.to_datetime(outlet_arch['date'])
    ids = outlet_arch["id_inv"]
    outlet_arch["id_inv"] = [int(i) for i in ids]
    outlet_arch = outlet_arch.rename(columns = {"id_inv":"id_spichlerz_wynajem", "date":"wizyta"})
    out = pd.merge(outlet_arch, tabela_g, on="wizyta", how="left")[["id_spichlerz_wynajem", "wizyta", "id_mieszkańca", "id_pracownika"]]
    out2 = pd.merge(out, spichlerz_wynajem[["id_spichlerz_wynajem", "id_gry", "cena_wynajem"]], on="id_spichlerz_wynajem", how="left")
    out2["cena_wynajem"] = [3* c for c in out2["cena_wynajem"]] 
    ##bo ustalamy 45% czyli 3* cenę najmu jako cenę kupna outletowej gry
    outlett = pd.merge(out2, klienci_mieszkancy, how="left", on = ["id_mieszkańca"])
    outlett["id_transakcji_outlet"] = range(1, len(outlett) + 1)
    outlet = outlett[["id_transakcji_outlet", "id_spichlerz_wynajem", "id_gry", "wizyta", "id_klienta", "id_pracownika"]]
    
    return klienci, spichlerz_wynajem, spichlerz_sklep, wynajem, sklep, outlet

In [60]:
tt = odwiedziny()

In [61]:
tab = format_tabel(tt)
klienci = tab[0]
spichlerz_wynajem = tab[1]
spichlerz_sklep = tab[2]
wynajem = tab[3]
sklep = tab[4]
outlet = tab[5]

In [57]:
outlet ##nie wiem czy zostawiać tu id inventorowe czy id gry

,id_transakcji_outlet,id_spichlerz_wynajem,id_gry,wizyta,id_klienta,id_pracownika
0,1,105,5,2022-06-09 12:54:06,28,2
1,2,1337,54,2022-06-10 11:26:54,39,1
2,3,867,35,2022-06-21 12:21:30,4426,2
3,4,764,31,2022-07-04 14:06:06,189,2
4,5,625,26,2022-07-16 16:58:54,289,2
...,...,...,...,...,...,...
59,60,150,6,2023-05-23 10:01:42,3988,2
60,61,2447,99,2023-05-26 12:52:00,4054,1
61,62,1063,43,2023-06-02 11:43:06,4174,1
62,63,468,20,2023-06-03 09:48:36,4203,2


In [51]:
tt[0][tt[0]["rodzaj"] == "outlet"]

,id_mieszkańca,wizyta,sprzedawca,ilosc,id_gry,rodzaj,return_date
0,1706,2022-06-10 11:20:42,2,1,1870.0,outlet,None
0,1932,2022-06-13 09:07:48,1,1,1488.0,outlet,None
0,3430,2022-06-18 10:41:00,1,1,46.0,outlet,None
0,3455,2022-07-02 15:28:54,2,1,1271.0,outlet,None
0,4331,2022-07-04 10:26:12,2,2,"1438.0, 2106.0",outlet,None
...,...,...,...,...,...,...,...
0,6390,2023-05-29 15:12:18,2,1,591.0,outlet,None
0,5892,2023-06-01 11:12:42,2,1,563.0,outlet,None
0,5722,2023-06-09 09:35:12,2,1,460.0,outlet,None
0,5260,2023-06-15 09:20:12,2,1,1065.0,outlet,None


In [52]:
klienci

,id_klienta,wizyta,imię,nazwisko,wiek,adres,telefon
0,1,2022-06-06 09:23:18,Daria,Wieczorek,31,Legendarna 30,608473254
1,2,2022-06-06 09:34:00,Siarhei,Nowicki,19,Jagodowa 35,692895403
2,3,2022-06-06 11:25:36,Artur,Szymczak,51,Zawilcowa 25,548485444
3,4,2022-06-06 12:39:30,Paula,Pawłowska,38,Południowa 69,532235551
4,5,2022-06-06 13:12:00,Renata,Orłowska,42,Pogodna 24,934148383
...,...,...,...,...,...,...,...
4584,4585,2023-06-20 15:51:42,Matylda,Piasecka,77,Niecała 80,560577578
4585,4586,2023-06-20 15:53:54,Olivia,Nowak,35,Cisowa 45,781656699
4586,4587,2023-06-20 15:55:24,Bolesław,Jabłoński,37,Zaciszna 12,521209799
4587,4588,2023-06-20 16:55:54,Jakub,Kubiak,35,Jarzębinowa 73,NULL


In [53]:
sklep

,id_transakcji_sklep,id_spichlerz_sklep,wizyta,id_klienta,id_pracownika
0,1,398,2022-06-06 09:30:48,3427,1
1,2,331,2022-06-06 09:30:48,3427,1
2,3,177,2022-06-06 09:34:00,2,1
3,4,2293,2022-06-06 09:34:00,2,1
4,5,2346,2022-06-06 09:34:00,2,1
...,...,...,...,...,...
6131,6132,1666,2023-06-20 13:27:30,4580,1
6132,6133,2063,2023-06-20 13:33:54,4581,2
6133,6134,990,2023-06-20 16:55:54,4588,1
6134,6135,1930,2023-06-20 16:55:54,4588,1


##  tabele sprzedawców

In [16]:
def payment(rola):
    if rola == "sprzedawca":
        return random.randint(3700, 4300)
    elif rola == "logistyk":
        return random.randint(5000, 6000)
    else:
        return random.randint(6000, 7000)

In [41]:
def sales(ul = ulice_jaworze):
    role = ["sprzedawca", "sprzedawca", "manager", "logistyk"]
    id_pracownika = [1,2,3,4]
    pay_s = payment("sprzedawca")
    pay_m = payment("manager")
    pay_l = payment("logistyk")
    pensje = [pay_s, pay_s, pay_m, pay_l]
    
    imiona = ["Misio", "Nati", "Alutka", "Fifol"]
    nazwiska = ["Bresiński", "Lach", "Myśliwiec", "Oszczepaliński"]
    wiek = [22, 22, 21, 21]
    r = random.sample(id_pracownika, k=4)
    
    adresy = [random.choice(ul) + ' ' + str(random.randint(1,100)) for _ in range(4)]
    telefony = [random.randint(500000000, 999999999) for _ in range(4)]
    
    one = pd.DataFrame({"id_pracownika" : r, "imię_pracownika" : imiona, 
                        "nazwisko_pracownika" : nazwiska, "wiek_pracownika": wiek,
                       "adres_pracownika" : adresy, "telefon_pracownika" : telefony})
    two = pd.DataFrame({"id_pracownika": id_pracownika, "rola": role, "pensja":pensje})
    
    pracownicy = pd.merge(one, two, on="id_pracownika")
    pracownicy = pracownicy.sort_values(by="id_pracownika")
    
    return pracownicy

In [90]:
pracownicy = sales()
pracownicy

,id_pracownika,imię_pracownika,nazwisko_pracownika,wiek_pracownika,adres_pracownika,telefon_pracownika,rola,pensja
3,1,Fifol,Oszczepaliński,21,Jeżynowa 86,683423555,sprzedawca,3744
2,2,Alutka,Myśliwiec,21,Mała 50,885110563,sprzedawca,3744
1,3,Nati,Lach,22,Średnia 53,589766415,manager,6675
0,4,Misio,Bresiński,22,Pelchrim 38,829475489,logistyk,5390
